In [ ]:
#%pip install nolds
%pip install arch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pywt
import itertools
import nolds
import warnings
warnings.filterwarnings("ignore")
import gc
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, LSTM, GRU, SimpleRNN, Conv1D,
                                     MaxPooling1D, Flatten, Input, Reshape,
                                     Lambda, concatenate, TimeDistributed)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import (mean_squared_error, mean_absolute_error, 
                             r2_score, explained_variance_score)
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model  # Para modelos GARCH

# ------------------------------------------------
# Configuración de directorios para resultados
# ------------------------------------------------
RESULTS_DIR = r'C:\Users\antonio-jose.martine\OneDrive - GFI\Documentos\Doctorado\articulo 8 peer review\results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# ------------------------------------------------
# Diebold–Mariano Test Implementation (Mejorada)
# ------------------------------------------------
def diebold_mariano_test(y_true, y_pred1, y_pred2, crit='MSE'):
    """
    Computes the Diebold-Mariano test statistic and p-value.
    
    Parameters:
    y_true : array-like, true values
    y_pred1 : array-like, predictions from model 1
    y_pred2 : array-like, predictions from model 2
    crit : str, loss function ('MSE', 'MAE', etc.)
    
    Returns:
    DM_stat : float, test statistic
    p_value : float, two-tailed p-value
    """
    e1 = y_true - y_pred1
    e2 = y_true - y_pred2
    
    if crit.lower() == 'mse':
        d_t = e1**2 - e2**2
    elif crit.lower() == 'mae':
        d_t = np.abs(e1) - np.abs(e2)
    else:
        d_t = e1**2 - e2**2

    mean_d = np.mean(d_t)
    var_d = np.var(d_t, ddof=1)  # sample variance

    n = len(d_t)
    DM_stat = mean_d / np.sqrt(var_d / n)
    
    # Calculate p-value using normal approximation
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(np.abs(DM_stat)))
    
    return DM_stat, p_value

# ------------------------------------------------
# 1. Load Data (con documentación de fuentes)
# ------------------------------------------------
def load_data():
    """
    Carga datos de mercado de múltiples fuentes:
    - VIX: CBOE Volatility Index (CBOE)
    - DIX: Dark Index (SpotGamma)
    - GEX: Gamma Exposure Index (SpotGamma)
    - SKEW: CBOE SKEW Index (CBOE)
    - PUTCALLRATIO: CBOE Put/Call Ratio (CBOE)
    
    Período seleccionado (2012-10-05 a 2025-03-27) cubre:
    - Períodos de mercado alcista (2013-2019)
    - Crisis por pandemia COVID-19 (2020)
    - Períodos de alta volatilidad (2022)
    - Recuperaciones de mercado (2023-2025)
    """
    df = pd.read_csv(r'C:\Users\antonio-jose.martine\OneDrive - GFI\Documentos\Doctorado\articulo 8 peer review\Data\merged_market_data_vix.csv',
                     parse_dates=['DATE'])
    df.columns = df.columns.str.upper()
    df.set_index('DATE', inplace=True)
    
    # Documentar distribución de datos
    data_distribution = df.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])
    data_distribution.to_csv(os.path.join(RESULTS_DIR, 'data_distribution.csv'))
    
    return df

# ------------------------------------------------
# 2. Fractal Filters (con documentación mejorada)
# ------------------------------------------------
def hurst_exponent(ts):
    """
    Calcula el exponente de Hurst para una serie temporal.
    
    Parámetros:
    ts : array-like, serie temporal
    
    Fórmula:
    H = poly[0] en la regresión log(τ) = H * log(lag) + c
    donde:
    τ = desviación estándar de las diferencias (ts[lag:] - ts[:-lag])
    lag = rango de rezagos (2-20)
    
    Retorna:
    H : float, exponente de Hurst
    """
    lags = range(2, 20)
    tau = []
    for lag in lags:
        if lag < len(ts):
            tau.append(np.std(ts[lag:] - ts[:-lag]))
        else:
            tau.append(np.nan)
    tau = [t for t in tau if not np.isnan(t)]
    if len(tau) < 2:
        return np.nan
    poly = np.polyfit(np.log(range(2, 2+len(tau))), np.log(tau), 1)
    return poly[0]

def apply_hurst(df, price_col='PRICE', window_size=100):
    df['HURST_PRICE'] = df[price_col].rolling(window=window_size).apply(hurst_exponent, raw=True)
    return df

def apply_wavelet_energy(segment, wavelet='db4', level=3):
    coeffs = pywt.wavedec(segment, wavelet, level=level)
    energy = [np.sum(c**2) for c in coeffs]
    return energy

def apply_wavelets(df, col_list=None, window=150):
    if col_list is None:
        col_list = ['PRICE', 'PUTCALLRATIO']
    wavelet_cols = []

    for col in col_list:
        feats = []
        for i in range(window, len(df)):
            segment = df[col].iloc[i - window:i]
            if segment.isnull().any():
                feats.append([np.nan]*4)
            else:
                feats.append(apply_wavelet_energy(segment, wavelet='db4', level=3))
        for j in range(4):
            new_col = f'WAVELET_{col}_L{j}'
            df[new_col] = [np.nan]*window + [x[j] for x in feats]
            wavelet_cols.append(new_col)

    return df, wavelet_cols

# ------------------------------------------------
# 3. Feature Preparation (con limpieza documentada)
# ------------------------------------------------
def prepare_features(df, features, target='PRICE', lookback=10, scale_method='MinMax'):
    """
    Prepara características para modelos de series temporales.
    
    Eliminamos filas con:
    - Valores faltantes irrecoverables
    - Marcas de tiempo faltantes (esenciales para la continuidad temporal)
    - Errores de formato en fechas
    
    Justificación: Los modelos de series temporales requieren secuencias continuas
    y completas para un aprendizaje efectivo.
    """
    if df.empty or len(df) <= lookback:
        return None, None, None, None
    
    try:
        # Guardar índice para referencia temporal
        time_index = df.index[lookback:]
        
        df_clean = df.dropna(subset=features+[target])
        if len(df_clean) <= lookback:
            return None, None, None, None

        if scale_method=='Standard':
            scaler = StandardScaler()
        else:
            scaler = MinMaxScaler()

        scaled_data = scaler.fit_transform(df_clean[features + [target]])
        
        X, y = [], []
        for i in range(lookback, len(scaled_data)):
            X.append(scaled_data[i - lookback:i, :-1])
            y.append(scaled_data[i, -1])

        X = np.array(X)
        y = np.array(y)
        return (X, y, scaler, time_index)
    except Exception as e:
        print("Error en prepare_features:", e)
        return None, None, None, None

# ------------------------------------------------
# Modelos de línea base (ARIMA + GARCH)
# ------------------------------------------------
def train_arima_baseline(y_series, p=1, d=0, q=1):
    model = ARIMA(y_series, order=(p, d, q))
    fitted = model.fit()
    return fitted

def train_garch_baseline(y_series, p=1, q=1):
    model = arch_model(y_series, vol='Garch', p=p, q=q)
    fitted = model.fit(disp='off')
    return fitted

# ------------------------------------------------
# GARCH corrected implementation for VIX forecasting
# ------------------------------------------------
def train_and_forecast_garch(y_series, forecast_horizon, p=1, q=1):
    """
    Train a GARCH model and properly forecast VIX levels.
    
    GARCH models predict volatility, not levels. This function:
    1. Creates returns data from VIX levels
    2. Fits a GARCH model to VIX returns
    3. Forecasts conditional volatility
    4. Converts volatility forecasts back to VIX level forecasts
    
    Parameters:
    -----------
    y_series : Series
        Input VIX time series
    forecast_horizon : int
        Number of steps to forecast
    p, q : int
        GARCH model parameters
        
    Returns:
    --------
    forecast_levels : array
        Forecasted VIX levels
    """
    # Calculate returns (percent changes) of VIX
    returns = y_series.pct_change().dropna()
    
    # Fit GARCH model on returns
    model = arch_model(returns, vol='Garch', p=p, q=q, mean='AR', lags=1)
    fitted = model.fit(disp='off')
    
    # Forecast volatility
    forecast = fitted.forecast(horizon=forecast_horizon)
    
    # Get variance forecasts for all horizons
    variance_forecasts = forecast.variance.values[-1, :]
    
    # Convert variance forecasts to VIX level forecasts
    # We use the last VIX value as our starting point
    last_value = y_series.iloc[-1]
    forecast_levels = np.zeros(forecast_horizon)
    
    # Generate VIX level forecasts using ARIMA(1,0,0) + GARCH dynamics
    forecast_levels[0] = last_value * (1 + fitted.params['mu'] + 
                                      np.random.normal(0, np.sqrt(variance_forecasts[0])))
    
    for i in range(1, forecast_horizon):
        # Use previous forecasted value and apply ARIMA(1,0,0) + GARCH process
        ar_component = fitted.params['mu'] + fitted.params.get('ar1', 0) * (
            (forecast_levels[i-1] / last_value) - 1)
        
        # Add the conditional volatility component
        vol_component = np.random.normal(0, np.sqrt(variance_forecasts[i]))
        
        # Calculate the next level
        forecast_levels[i] = last_value * (1 + ar_component + vol_component)
    
    # Ensure no negative VIX values (which would be unrealistic)
    forecast_levels = np.maximum(forecast_levels, 1.0)
    
    return forecast_levels

# ------------------------------------------------
# CapsNet Implementation
# ------------------------------------------------
def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule.
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

def build_capsule_model(input_shape):
    """
    Build a simple CapsNet model for time series regression.
    """
    input_layer = Input(shape=input_shape)
    
    # Flatten the input for dense layers
    x = Flatten()(input_layer)
    
    # Primary capsules
    x = Dense(128, activation='relu')(x)
    x = Reshape((8, 16))(x)  # 8 capsules with 16 dimensions each
    
    # Apply squash activation
    primary_caps = Lambda(squash)(x)
    
    # Flatten for final prediction
    x = Flatten()(primary_caps)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)
    
    model = Model(inputs=input_layer, outputs=output)
    return model

# ------------------------------------------------
# Missing utility functions
# ------------------------------------------------
def calculate_metrics(y_true, y_pred):
    """
    Calculate various regression metrics.
    """
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    evs = explained_variance_score(y_true, y_pred)
    
    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'explained_variance': evs
    }

def compare_models_dm(pred_df, group_cols=['features', 'fractal']):
    """
    Compare models using Diebold-Mariano test.
    """
    results = []
    
    # Group by features and fractal type
    for group_vals, group_data in pred_df.groupby(group_cols):
        models = group_data['model'].unique()
        
        for i, model1 in enumerate(models):
            for model2 in models[i+1:]:
                model1_data = group_data[group_data['model'] == model1]
                model2_data = group_data[group_data['model'] == model2]
                
                # Ensure same dates for comparison
                common_dates = set(model1_data['date']).intersection(set(model2_data['date']))
                if len(common_dates) < 10:  # Need sufficient data
                    continue
                
                model1_subset = model1_data[model1_data['date'].isin(common_dates)].sort_values('date')
                model2_subset = model2_data[model2_data['date'].isin(common_dates)].sort_values('date')
                
                if len(model1_subset) == len(model2_subset):
                    dm_stat, p_value = diebold_mariano_test(
                        model1_subset['y_true'].values,
                        model1_subset['y_pred'].values,
                        model2_subset['y_pred'].values
                    )
                    
                    result = {
                        'model1': model1,
                        'model2': model2,
                        'dm_statistic': dm_stat,
                        'p_value': p_value,
                        'significant': p_value < 0.05
                    }
                    
                    # Add group information
                    for j, col in enumerate(group_cols):
                        result[col] = group_vals[j] if isinstance(group_vals, tuple) else group_vals
                    
                    results.append(result)
    
    return pd.DataFrame(results)

# ------------------------------------------------
# 4. Build Models (con parámetros documentados)
# ------------------------------------------------
def build_model(model_type, input_shape):
    """
    Construye modelos con parámetros completamente documentados:
    
    ANN: 
        Capas: [128, 64, 32] neuronas, activación ReLU
    RNN: 
        2 capas RNN [128, 64] neuronas
    LSTM: 
        2 capas LSTM [128, 64] neuronas
    GRU: 
        2 capas GRU [128, 64] neuronas
    CNN: 
        Conv1D(128)-MaxPooling-Conv1D(64)-MaxPooling-Flatten
    CNN-LSTM: 
        Conv1D(64)-MaxPooling-LSTM(128)-LSTM(64)
    CapsNet: 
        Arquitectura basada en cápsulas con parámetros específicos
    """
    params = {
        'batch_size': 64,
        'epochs': 100,
        'optimizer': Adam(learning_rate=0.001),
        'loss': 'mse'
    }
    
    if model_type == 'CapsNet':
        print(f"Construyendo CapsNet: input_shape={input_shape}, dim_capsule=16, num_caps=10, rutings=3")
        model = build_capsule_model(input_shape)
        model.compile(optimizer=params['optimizer'], loss=params['loss'])
        return model, params
    
    model = Sequential()
    if model_type == 'ANN':
        print(f"Construyendo ANN: 3 capas densas [128, 64, 32], input_shape={input_shape}")
        model.add(Input(shape=input_shape))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(1))
    
    elif model_type == 'RNN':
        print(f"Construyendo SimpleRNN: 2 capas [128, 64], input_shape={input_shape}")
        model.add(SimpleRNN(128, return_sequences=True, input_shape=input_shape))
        model.add(SimpleRNN(64))
        model.add(Dense(1))
    
    elif model_type == 'LSTM':
        print(f"Construyendo LSTM: 2 capas [128, 64], input_shape={input_shape}")
        model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
        model.add(LSTM(64))
        model.add(Dense(1))
    
    elif model_type == 'GRU':
        print(f"Construyendo GRU: 2 capas [128, 64], input_shape={input_shape}")
        model.add(GRU(128, return_sequences=True, input_shape=input_shape))
        model.add(GRU(64))
        model.add(Dense(1))
    
    elif model_type == 'CNN':
        print(f"Construyendo CNN: Conv1D(128)-Conv1D(64), input_shape={input_shape}")
        model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(64, kernel_size=3, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Flatten())
        model.add(Dense(1))
    
    elif model_type == 'CNN_LSTM':
        print(f"Construyendo CNN-LSTM: Conv1D(64)-LSTM(128)-LSTM(64), input_shape={input_shape}")
        model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(LSTM(128, return_sequences=True))
        model.add(LSTM(64))
        model.add(Dense(1))

    model.compile(optimizer=params['optimizer'], loss=params['loss'])
    return model, params

# ------------------------------------------------
# 5. Entrenamiento con validación cruzada temporal
# ------------------------------------------------
def train_with_cv(X, y, model_type, time_index, n_splits=5, epochs=100, batch_size=64):
    """
    Entrenamiento con validación cruzada temporal y optimización de hiperparámetros
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    all_preds = []
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        test_dates = time_index[test_index]
        
        model, params = build_model(model_type, X_train.shape[1:])
        
        # Callback para early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        history = model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=0
        )
        
        y_pred = model.predict(X_test).flatten()
        metrics = calculate_metrics(y_test, y_pred)
        fold_metrics.append(metrics)
        
        # Guardar predicciones para análisis posterior
        for i in range(len(y_test)):
            all_preds.append({
                'date': test_dates[i],
                'y_true': y_test[i],
                'y_pred': y_pred[i],
                'model': model_type
            })
        
        # Liberar memoria
        del model
        tf.keras.backend.clear_session()
        gc.collect()
    
    # Calcular métricas promedio
    avg_metrics = {}
    for key in fold_metrics[0].keys():
        avg_metrics[key] = np.mean([m[key] for m in fold_metrics])
    
    return avg_metrics, pd.DataFrame(all_preds)

# ------------------------------------------------
# 6. Visualización de resultados
# ------------------------------------------------
def plot_results(pred_df, title, filename):
    plt.figure(figsize=(15, 8))
    
    # Plot de predicciones vs valores reales
    plt.subplot(2, 2, 1)
    plt.plot(pred_df['date'], pred_df['y_true'], 'b-', label='Valor Real')
    plt.plot(pred_df['date'], pred_df['y_pred'], 'r--', label='Predicción')
    plt.title(f'Predicciones vs Reales: {title}')
    plt.xlabel('Fecha')
    plt.ylabel('VIX')
    plt.legend()
    plt.grid(True)
    
    # Plot de errores
    plt.subplot(2, 2, 2)
    errors = pred_df['y_true'] - pred_df['y_pred']
    plt.plot(pred_df['date'], errors, 'g-')
    plt.title('Errores de Predicción')
    plt.xlabel('Fecha')
    plt.ylabel('Error')
    plt.grid(True)
    
    # Histograma de errores
    plt.subplot(2, 2, 3)
    sns.histplot(errors, kde=True)
    plt.title('Distribución de Errores')
    plt.xlabel('Error')
    plt.ylabel('Frecuencia')
    
    # Boxplot por modelo
    plt.subplot(2, 2, 4)
    sns.boxplot(x='model', y=errors, data=pred_df)
    plt.title('Distribución de Errores por Modelo')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, filename))
    plt.close()

# ------------------------------------------------
# 7. Análisis por condición de mercado
# ------------------------------------------------
def analyze_market_conditions(pred_df, vix_thresholds=[20, 30]):
    """
    Analiza el rendimiento del modelo en diferentes condiciones de mercado:
    - Mercado tranquilo: VIX < 20
    - Mercado volátil: 20 ≤ VIX < 30
    - Mercado extremo: VIX ≥ 30
    """
    results = []
    
    # Clasificar condiciones de mercado
    conditions = []
    for vix in pred_df['y_true']:
        if vix < vix_thresholds[0]:
            conditions.append('Tranquilo')
        elif vix < vix_thresholds[1]:
            conditions.append('Volátil')
        else:
            conditions.append('Extremo')
    
    pred_df['market_condition'] = conditions
    
    # Calcular métricas por condición
    for condition in ['Tranquilo', 'Volátil', 'Extremo']:
        cond_df = pred_df[pred_df['market_condition'] == condition]
        if len(cond_df) > 0:
            metrics = calculate_metrics(cond_df['y_true'], cond_df['y_pred'])
            metrics['condition'] = condition
            metrics['count'] = len(cond_df)
            results.append(metrics)
    
    return pd.DataFrame(results)

# ------------------------------------------------
# Función principal actualizada
# ------------------------------------------------
def benchmark_all_combinations():
    base_cols = ['DIX', 'GEX', 'SKEW', 'PUTCALLRATIO']
    model_types = ['ARIMA', 'GARCH', 'ANN', 'RNN', 'LSTM', 'GRU', 'CNN', 'CNN_LSTM', 'CapsNet']

    results = []
    all_predictions = []
    
    # Documentar parámetros de modelos
    with open(os.path.join(RESULTS_DIR, 'model_parameters.txt'), 'w') as f:
        f.write("Parámetros de Modelos:\n")
        f.write("="*50 + "\n")
        for model_type in model_types:
            if model_type not in ['ARIMA', 'GARCH']:
                model, params = build_model(model_type, (10, 5))  # Ejemplo de forma
                f.write(f"\n{model_type}:\n")
                for key, value in params.items():
                    f.write(f"{key}: {value}\n")
                f.write("-"*30 + "\n")
    
    # Cada combinación de características
    for r in range(1, len(base_cols)+1):
        for combo in itertools.combinations(base_cols, r):
            combo_list = list(combo)
            for fractal in ['none', 'hurst', 'wavelet']:
                df = load_data()
                extra_feats = []
                
                if fractal == 'hurst':
                    df = apply_hurst(df, price_col='PRICE', window_size=100)
                    extra_feats = ['HURST_PRICE']
                elif fractal == 'wavelet':
                    df, wave_cols = apply_wavelets(df, col_list=['PRICE', 'PUTCALLRATIO'], window=150)
                    extra_feats = wave_cols

                all_features = combo_list + extra_feats
                df.dropna(subset=all_features+['VIX'], inplace=True)
                
                # Preparar características con índice temporal
                X, y, scaler, time_index = prepare_features(
                    df, features=all_features, target='VIX', lookback=10
                )
                
                if X is None or y is None:
                    continue
                
                for m in model_types:
                    try:
                        if m in ['ARIMA', 'GARCH']:
                            # Modelos estadísticos
                            vix_series = df['VIX'].dropna()
                            if len(vix_series) < 30:
                                continue
                                
                            test_split = int(len(vix_series)*0.8)
                            vix_train = vix_series.iloc[:test_split]
                            vix_test  = vix_series.iloc[test_split:]
                            
                            if m == 'ARIMA':
                                model = train_arima_baseline(vix_train)
                                forecast = model.forecast(steps=len(vix_test))
                                y_pred = forecast.values
                            else:  # GARCH - CORREGIDO
                                # Usar el nuevo método de pronóstico GARCH
                                y_pred = train_and_forecast_garch(vix_train, forecast_horizon=len(vix_test))
                            
                            # Evaluar
                            metrics = calculate_metrics(vix_test.values, y_pred)
                            metrics['train_loss'] = np.nan
                            metrics['val_loss'] = np.nan
                            
                            # Guardar predicciones
                            for i, datex in enumerate(vix_test.index):
                                all_predictions.append({
                                    'date': datex,
                                    'features': '+'.join(combo_list),
                                    'fractal': fractal,
                                    'model': m,
                                    'y_true': vix_test.iloc[i],
                                    'y_pred': y_pred[i]
                                })
                        else:
                            # Modelos de deep learning con CV
                            metrics, pred_df = train_with_cv(
                                X, y, m, time_index, n_splits=5
                            )
                            
                            # Guardar predicciones
                            pred_df['features'] = '+'.join(combo_list)
                            pred_df['fractal'] = fractal
                            all_predictions.extend(pred_df.to_dict('records'))
                        
                        # Guardar resultados
                        result_dict = {
                            'features': '+'.join(combo_list),
                            'model': m,
                            'fractal': fractal,
                            **metrics
                        }
                        results.append(result_dict)
                        
                        print(f"✅ {m} | {combo_list} + {fractal} => RMSE={metrics.get('rmse','N/A'):.4f}, R2={metrics.get('r2','N/A'):.4f}")
                    
                    except Exception as e:
                        print(f"❌ Error en {m} | {combo_list} + {fractal}: {str(e)}")
    
    # Guardar resultados
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(RESULTS_DIR, 'combo_results_enhanced.csv'), index=False)
    
    # Guardar todas las predicciones
    all_preds_df = pd.DataFrame(all_predictions)
    all_preds_df.to_csv(os.path.join(RESULTS_DIR, 'all_predictions.csv'), index=False)
    
    # Visualización de resultados
    plot_results(all_preds_df, 'Todos los Modelos', 'all_models_performance.png')
    
    # Análisis por condición de mercado
    market_analysis = analyze_market_conditions(all_preds_df)
    market_analysis.to_csv(os.path.join(RESULTS_DIR, 'market_condition_analysis.csv'), index=False)
    
    # Comparaciones DM
    dm_df = compare_models_dm(all_preds_df, group_cols=['features','fractal'])
    dm_df.to_csv(os.path.join(RESULTS_DIR, 'dm_results.csv'), index=False)
    
    print("✅ Todos los análisis completados y resultados guardados")

if __name__ == '__main__':
    benchmark_all_combinations()
    gc.collect()


Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 5)
Construyendo SimpleRNN: 2 capas [128, 64], input_shape=(10, 5)
Construyendo LSTM: 2 capas [128, 64], input_shape=(10, 5)
Construyendo GRU: 2 capas [128, 64], input_shape=(10, 5)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Construyendo CNN: Conv1D(128)-Conv1D(64), input_shape=(10, 5)
Construyendo CNN-LSTM: Conv1D(64)-LSTM(128)-LSTM(64), input_shape=(10, 5)
Construyendo CapsNet: input_shape=(10, 5), dim_capsule=16, num_caps=10, rutings=3

✅ ARIMA | ['DIX'] + none => RMSE=3.4393, R2=0.3624
✅ GARCH | ['DIX'] + none => RMSE=536.3863, R2=-15506.7327
Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Construyendo ANN: 3 capas densas [128, 64, 32], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
✅ ANN | ['DIX'] + none => RMSE=0.0950, R2=-1.0436
Construyendo SimpleRNN: 2 capas [128, 64], input_shape=(10, 1)
17/17 ━━━━━━━━━━━━━━━

KeyboardInterrupt: 